# MD5

In [5]:
import struct
import math

# Constants for MD5
s = [7, 12, 17, 22] * 4 + [5, 9, 14, 20] * 4 + [4, 11, 16, 23] * 4 + [6, 10, 15, 21] * 4
K = [int(abs(math.sin(i + 1)) * (2**32)) & 0xFFFFFFFF for i in range(64)]
init_values = [0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476]

def left_rotate(x, c):
    return (x << c) & 0xFFFFFFFF | (x >> (32 - c))

def md5(message):
    message = bytearray(message.encode('utf-8'))
    orig_len_in_bits = (8 * len(message)) & 0xFFFFFFFFFFFFFFFF
    message.append(0x80)
    while len(message) % 64 != 56:
        message.append(0)
    message += orig_len_in_bits.to_bytes(8, byteorder='little')

    A, B, C, D = init_values

    for chunk_ofst in range(0, len(message), 64):
        a, b, c, d = A, B, C, D
        chunk = message[chunk_ofst:chunk_ofst + 64]
        M = struct.unpack('<16I', chunk)

        for i in range(64):
            if 0 <= i <= 15:
                f = (b & c) | (~b & d)
                g = i
            elif 16 <= i <= 31:
                f = (d & b) | (~d & c)
                g = (5 * i + 1) % 16
            elif 32 <= i <= 47:
                f = b ^ c ^ d
                g = (3 * i + 5) % 16
            elif 48 <= i <= 63:
                f = c ^ (b | ~d)
                g = (7 * i) % 16

            f = (f + a + K[i] + M[g]) & 0xFFFFFFFF
            a, d, c, b = d, (b + left_rotate(f, s[i])) & 0xFFFFFFFF, b, c

        A = (A + a) & 0xFFFFFFFF
        B = (B + b) & 0xFFFFFFFF
        C = (C + c) & 0xFFFFFFFF
        D = (D + d) & 0xFFFFFFFF

    return '{:08x}{:08x}{:08x}{:08x}'.format(A, B, C, D)

# Example usage
message = "빅데이터 핀테크 8기 화이팅"
hash_value = md5(message)
print(f"MD5('{message}') = {hash_value}")


MD5('빅데이터 핀테크 8기  화이팅') = a23d2e5cdf9b57123175b9fce0aa457b


# DL-based 해쉬

In [2]:
import random
from sympy import isprime, mod_inverse

def generate_prime_pair(bits=128):
    while True:
        q = random.getrandbits(bits)
        if isprime(q) and isprime(2 * q + 1):
            p = 2 * q + 1
            return p, q

def find_primitive_roots(p):
    def is_primitive_root(g, p):
        if pow(g, 2, p) == 1 or pow(g, (p - 1) // 2, p) == 1:
            return False
        return True
    
    g = random.randint(2, p-2)
    while not is_primitive_root(g, p):
        g = random.randint(2, p-2)
    
    h = random.randint(2, p-2)
    while not is_primitive_root(h, p) or h == g:
        h = random.randint(2, p-2)
    
    return g, h

def hash_function(p, q, g, h, m): # m = u + vq
    u = m % q
    v = m // q
    return (pow(g, u, p) * pow(h, v, p)) % p

# Reduce the bit size for testing purposes
p, q = generate_prime_pair(bits=64)

g, h = find_primitive_roots(p)

m = random.randint(0, q*q - 1)
# m = 284748387483939874574583993947757889

hash_value = hash_function(p, q, g, h, m)

print(f"Prime p: {p}")
print(f"Prime q: {q}")
print(f"Primitive root g: {g}")
print(f"Primitive root h: {h}")
print(f"Message m: {m}")
print(f"Hash value: {hash_value}")


Prime p: 31596311500983607319
Prime q: 15798155750491803659
Primitive root g: 16524517190720207100
Primitive root h: 24271410953738786705
Message m: 113842287193006788823202072844940797900
Hash value: 9133486915914586743


# MD5-Round function

In [1]:
import struct
import math

# MD5 알고리즘에 필요한 상수
s = [7, 12, 17, 22]  # 각 단계에서 비트 회전량
K = [int((abs(math.sin(i + 1)) * 2**32)) & 0xFFFFFFFF for i in range(4)]

def left_rotate(x, c):
    """
    x를 c만큼 왼쪽으로 회전시킵니다. 회전 시 32비트 크기를 유지합니다.
    """
    return (x << c | x >> (32 - c)) & 0xFFFFFFFF

def md5_one_round_educational(message):
    """
    MD5 알고리즘의 한 라운드를 수행하여 각 단계별 결과를 출력합니다.
    """
    # 입력 메시지를 바이트 배열로 변환하고, 패딩을 추가
    message = bytearray(message.encode('utf-8'))
    orig_len_in_bits = (8 * len(message)) & 0xffffffffffffffff
    message.append(0x80)
    while len(message) % 64 != 56:
        message.append(0)
    message += orig_len_in_bits.to_bytes(8, byteorder='little')

    # MD5 초기화 변수 설정
    A = 0x67452301
    B = 0xEFCDAB89
    C = 0x98BADCFE
    D = 0x10325476

    # 메시지를 512비트(64바이트) 블록으로 분할
    chunk = message[:64]
    M = [struct.unpack('<I', chunk[i:i+4])[0] for i in range(0, 64, 4)]

    # 초기 변수 설정
    a, b, c, d = A, B, C, D

    print(f"Initial values: A = {A:08x}, B = {B:08x}, C = {C:08x}, D = {D:08x}")

    # 한 라운드의 첫 4단계 수행
    for i in range(4):
        # F 함수는 b, c, d의 값에 따라 연산 수행
        F = (b & c) | (~b & d)
        g = i  # 이 단계에서 인덱스 g는 i와 같다.

        # F에 a, K[i], M[g]를 더하고 비트 회전
        F = (F + a + M[g] + K[i]) & 0xFFFFFFFF
        a, d, c, b = d, (b + left_rotate(F, s[i])) & 0xFFFFFFFF, b, c

        # 각 단계 후 변수의 값을 출력합니다.
        print(f"After step {i+1}: A = {a:08x}, B = {b:08x}, C = {c:08x}, D = {d:08x}")

    return a, b, c, d

# 테스트
if __name__ == "__main__":
    test_string = "빅데이터 핀테크"
    md5_one_round_educational(test_string)



Initial values: A = 67452301, B = efcdab89, C = 98badcfe, D = 10325476
After step 1: A = 10325476, B = 98badcfe, C = efcdab89, D = 67fcdcea
After step 2: A = 67fcdcea, B = efcdab89, C = 98badcfe, D = cd08f569
After step 3: A = cd08f569, B = 98badcfe, C = efcdab89, D = 209135df
After step 4: A = 209135df, B = efcdab89, C = 98badcfe, D = 58e8d694


# VSH 

In [3]:
import hashlib
import sympy

def vsh_hash(message, prime):
    """
    VSH 해시 함수를 구현.
    
    :param message: 해시할 메시지 (바이트 문자열)
    :param prime: 매우 큰 소수
    :return: 해시 값 (정수)
    """
    # 메시지를 블록으로 분할 (각 블록은 256비트)
    block_size = 32  # 256비트는 32바이트
    blocks = [message[i:i + block_size] for i in range(0, len(message), block_size)]
    
    hash_value = 1
    block_count = 0
    
    for block in blocks:
        block_count += 1
        # 블록을 정수로 변환
        block_int = int.from_bytes(block, byteorder='big')
        print(f'Block {block_count}: {block} -> Integer: {block_int}')
        
        # 블록의 소인수 분해
        factors = sympy.factorint(block_int)
        print(f'Block {block_count} factors: {factors}')
        
        # 소수의 지수를 곱하여 해시 값 계산
        for prime_factor, exponent in factors.items():
            hash_value = (hash_value * pow(prime_factor, exponent, prime)) % prime
            print(f'Intermediate hash_value after factoring {prime_factor}^{exponent}: {hash_value}')
    
    return hash_value

# 예제 메시지
message = b'This is an example message for VSH algorithm. It is designed for understanding.'

# 매우 큰 소수 (예제에서는 2^256에 가장 가까운 소수를 사용)
prime = sympy.nextprime(2**256)

# VSH 해시 계산
hash_value = vsh_hash(message, prime)

print(f'VSH hash value: {hash_value}')



Block 1: b'This is an example message for V' -> Integer: 38178759162904981154311987488508316457765032398410928252518357528642237636694
Block 1 factors: {2: 1, 223757400049: 1, 85312841395512109761625315479776390726623892901870031445071971803: 1}
Intermediate hash_value after factoring 2^1: 2
Intermediate hash_value after factoring 223757400049^1: 447514800098
Intermediate hash_value after factoring 85312841395512109761625315479776390726623892901870031445071971803^1: 38178759162904981154311987488508316457765032398410928252518357528642237636694
Block 2: b'SH algorithm. It is designed for' -> Integer: 37669402903489356249222610683602484085829906402471994911811268713869222506354
Block 2 factors: {2: 1, 19: 2, 38671: 1, 439429: 1, 4768980766637: 1, 26048758367: 1, 635320852862040011: 1, 38901980604511929528067: 1}
Intermediate hash_value after factoring 2^1: 76357518325809962308623974977016632915530064796821856505036715057284475273388
Intermediate hash_value after factoring 19^2: 6546877136

# Kaccak

In [5]:
import hashlib

def keccak_f(state):
    # Keccak-f 퍼뮤테이션 함수 (간단히 설명하기 위해 임의로 설정)
    return hashlib.sha3_512(state).digest()

def sponge_construction(message, rate, capacity, output_length):
    # 초기 상태 설정
    state_size = rate + capacity
    state = bytearray(state_size // 8)
    
    # 메시지를 블록으로 나누기
    message_blocks = [message[i:i + rate // 8] for i in range(0, len(message), rate // 8)]
    
    # 흡수 단계
    for block in message_blocks:
        for i in range(len(block)):
            state[i] ^= block[i]
        state = keccak_f(state)
    
    # 짜내기 단계
    output = bytearray()
    while len(output) < output_length:
        output.extend(state[:rate // 8])
        state = keccak_f(state)
    
    return output[:output_length]

# 예제 메시지
message = b'This is an example message for Keccak sponge construction.'

# Sponge Construction 파라미터
rate = 576  # 비트
capacity = 1024  # 비트
output_length = 64  # 바이트

# Keccak 해시 계산
hash_value = sponge_construction(message, rate, capacity, output_length)

print(f'Keccak hash value: {hash_value.hex()}')


Keccak hash value: f15c98a3e00f27c8053d8ea4688d102d59f8777f2e86741bfc7f339591b0b6b2d7f781258aec61d2c30dd33d6212cb526d526dd7457b70296738ef2725cdd0eb


# Keccak-f(permutation)

In [27]:
import numpy as np

# Keccak-f permutation constants (첫 번째 라운드만 사용)
ROUND_CONSTANTS = [0x0000000000000001]

# Keccak-f permutation
def keccak_f(state):
    def rol(value, shift, size=64):
        return np.bitwise_or(
            np.left_shift(np.uint64(value), np.uint64(shift)) & np.uint64((1 << size) - 1),
            np.right_shift(np.uint64(value), np.uint64(size - shift))
        )

    state = np.array(state, dtype=np.uint64).reshape(5, 5)

    # 한 라운드만 수행
    round_constant = ROUND_CONSTANTS[0]
    print(f"Initial state:\n{state}")

    # Theta step
    C = np.zeros(5, dtype=np.uint64)
    D = np.zeros(5, dtype=np.uint64)
    
    for x in range(5):
        C[x] = np.bitwise_xor.reduce(state[x, :])
    print(f"After computing C in Theta:\n{C}")
    
    for x in range(5):
        D[x] = np.bitwise_xor(C[(x - 1) % 5], rol(C[(x + 1) % 5], 1))
    print(f"After computing D in Theta:\n{D}")
    
    for x in range(5):
        for y in range(5):
            state[x, y] = np.bitwise_xor(state[x, y], D[x])
    print(f"After Theta step:\n{state}")

    # Rho and Pi steps
    B = np.zeros((5, 5), dtype=np.uint64)
    for x in range(5):
        for y in range(5):
            B[y, (2 * x + 3 * y) % 5] = rol(state[x, y], (x + 1) * (y + 1) % 64)
    print(f"After Rho and Pi steps:\n{B}")

    # Chi step
    for x in range(5):
        for y in range(5):
            state[x, y] = np.bitwise_xor(B[x, y], np.bitwise_and(~B[(x + 1) % 5, y], B[(x + 2) % 5, y]))
    print(f"After Chi step:\n{state}")

    # Iota step
    state[0, 0] = np.bitwise_xor(state[0, 0], np.uint64(round_constant))
    print(f"After Iota step:\n{state}\n")

    return state.flatten()

# 초기 상태 (난수로 설정)
np.random.seed(0)  # 난수 생성을 위해 시드 고정
state = np.random.randint(0, 2**64, size=25, dtype=np.uint64)


# Keccak-f 퍼뮤테이션 수행 (한 라운드만)
new_state = keccak_f(state)

print(f'Final state:\n{new_state}')


Initial state:
[[10123822223749065263 13192915183495924928 11119021634053362427
  10051320526890433895  7815051614634499795]
 [11914643441472427762  8072059273656089175 16450308335489826264
  17776440383103618444  7073247613084517057]
 [14604749071893285415  9756389261445154990 10478572691744721745
  17074244212226605593  1310383996493862984]
 [ 1607251845270378388   372963635737465808 15359125155679313605
  14354458544163391311 16048891388304951744]
 [18052322119257055331 14741873504861846705  8512791703084466973
  14398221948235222163  2181778028650783143]]
After computing C in Theta:
[ 5057205178604951456 11929042644624072240  2484867697141011337
 16125661665629017102 11071931888724112123]
After computing D in Theta:
[15186020631195143834   204563392917024946  1882450377206068781
  1239273613507311230  6022510591090146126]
After Theta step:
[[ 6827632830485849269  7325442600645821018  5256024080641006689
   6467982212855149565 13748136470730903625]
 [12074027760934774336  8274087190